In [1]:
import pandas as pd
from datetime import datetime
from datetime import timedelta     

In [2]:
NGR=pd.read_csv('playerstats.csv')

In [3]:
# Deletando a última linha do DataFrame
NGR = NGR.drop(NGR.index[-1])

In [4]:
NGR["First deposit"] =pd.to_datetime(NGR["First deposit"], errors = 'coerce')

In [5]:
NGR["Advertiser"].unique()

array(['3snet18432', 'defaultgalera.bet', 'mobytize', 'invivoads',
       'AlfaleadsCPA', 'Affunlim', 'IdentiBRDisplay', 'rspcom', 'msbtt',
       'ingoogleads', 'marketin', 'crmbonus', 'Skores', 'ally0004',
       'ally0034', 'ally0032', 'ally0031', 'ally0011', 'seoglr', 'gohike',
       'nsnwab', 'intaboola', 'liqes', 'LiqesGA_Mines', 'ally0005',
       'betandealpt', 'knownmedia', 'bccph22ppcbr', 'bc18verd3',
       'rubju300', 'marcus.igoalcorp', '3snet1010', 'ally0021',
       'intimads', 'supera088', 'GaleraLimayPPC', 'julia.advertise.pro',
       'allyinf0011', 'breakmedia', 'bm22', 'jompa', 'PropellerAds2',
       'leadbitolga', 'dmidd47', 'samir', 'agente25', 'bwise_influencers',
       'liqesGA', 'glaununes', 'equadratBR', 'g1odds', '365scores',
       'EonF.Galera', 'ally0012', 'premierleads', 'scalemedia',
       'jafe1000', 'LeadsGlobal', 'MoveUpGal01', 'ally0038', '3snet13678',
       'Ishai', 'vmmeightroom', 'gaffiliation2022', 'Myaffdeal',
       'bcpar15', 'thago', 'al

In [6]:
NGR["Calculated_NGR"] = NGR['Casino Real Bets'] - NGR['Casino Real Wins'] -NGR['Redeemed bonus initial amounts'] - NGR['Redeemed bonus pending winnings']

In [7]:
NGR['First deposit date abreviated'] =NGR["First deposit"].dt.strftime('%m/%Y')

In [8]:
NGR.columns.values

array(['Username', 'Country', 'Advertiser', 'Signed up', 'First deposit',
       'Last deposit date', 'Wants mail', 'Deposits', 'Deposit count',
       'Average deposit', 'Approved Withdrawals',
       'Approved Withdrawal count', 'Casino NGR', 'Casino bonuses',
       'Casino bonus count', 'Redeemed bonus initial amounts',
       'Redeemed bonus pending winnings', 'Poker bonus reverses',
       'Casino progressive share', 'Casino progressive wins',
       'Casino Bets', 'Casino Wins', 'Casino Real Bets',
       'Casino Real Wins', 'Casino game count', 'Casino bonuses removed',
       'Frozen', 'Sportsbook wins', 'Sportsbook bets',
       'Sportsbook gross income', 'Calculated_NGR',
       'First deposit date abreviated'], dtype=object)

In [9]:
output = NGR.groupby('First deposit date abreviated')[["Calculated_NGR","Deposits","Deposit count"]].sum().reset_index()

In [10]:
output.loc[len(output)] = ["Total",output["Calculated_NGR"].sum(),output["Deposits"].sum(),output["Deposit count"].sum()]



In [11]:
import requests
# Conversão para DataFrame
df = pd.DataFrame(output)

# Renomeando colunas
df.rename(columns={
    "First deposit date abreviated": "Mês",
    "Calculated_NGR": "NGR",
    "Deposits": "Depósitos"
}, inplace=True)

# Fazendo a requisição para a API de câmbio (exemplo com Exchange Rates API)
api_key = "YOUR_API_KEY"  # Substitua pela sua chave de API
url = f"https://api.exchangerate-api.com/v4/latest/BRL"
response = requests.get(url)
data = response.json()

# Obter a taxa de conversão de BRL para USD
exchange_rate = data["rates"]["USD"]

# Adicionando colunas em USD
df["NGR_USD"] = df["NGR"] * exchange_rate
df["Depósitos_USD"] = df["Depósitos"] * exchange_rate

# Formatando os valores em R$ e em US$
df["NGR"] = df["NGR"].apply(lambda x: f'R${x:,.2f}'.replace(",", "X").replace(".", ",").replace("X", "."))
df["Depósitos"] = df["Depósitos"].apply(lambda x: f'R${x:,.2f}'.replace(",", "X").replace(".", ",").replace("X", "."))
df["NGR_USD"] = df["NGR_USD"].apply(lambda x: f'${x:,.2f}')
df["Depósitos_USD"] = df["Depósitos_USD"].apply(lambda x: f'${x:,.2f}')

df

,Mês,NGR,Depósitos,Deposit count,NGR_USD,Depósitos_USD
0,01/2022,"R$-143,23","R$724,00",11,$-24.78,$125.25
1,01/2023,"R$-9.893,98","R$23.659,34",344,"$-1,711.66","$4,093.07"
2,01/2024,"R$-973,77","R$26.395,43",467,$-168.46,"$4,566.41"
3,02/2022,"R$1.417,58","R$4.770,62",72,$245.24,$825.32
4,02/2023,"R$3.969,31","R$17.028,97",260,$686.69,"$2,946.01"
5,02/2024,"R$12.485,59","R$27.603,13",501,"$2,160.01","$4,775.34"
6,03/2022,"R$4.750,62","R$16.255,16",139,$821.86,"$2,812.14"
7,03/2023,"R$5.533,01","R$16.669,31",263,$957.21,"$2,883.79"
8,03/2024,"R$12.516,57","R$42.782,24",454,"$2,165.37","$7,401.33"
9,04/2022,"R$16.033,38","R$19.031,12",269,"$2,773.77","$3,292.38"


In [12]:
import pandas as pd
import requests
# Supondo que df já está criado conforme o exemplo dado

# Conversão para DataFrame
df = pd.DataFrame(output)

# Renomeando colunas
df.rename(columns={
    "First deposit date abreviated": "Mês",
    "Calculated_NGR": "NGR",
    "Deposits": "Depósitos"
}, inplace=True)

# Separando o ano e mês
df['Ano'] = df['Mês'].str.split('/').str[1]
df['Mês_num'] = df['Mês'].str.split('/').str[0]


# Fazendo a requisição para a API de câmbio (exemplo com Exchange Rates API)
api_key = "YOUR_API_KEY"  # Substitua pela sua chave de API
url = f"https://api.exchangerate-api.com/v4/latest/BRL"
response = requests.get(url)
data = response.json()

# Obter a taxa de conversão de BRL para USD
exchange_rate = data["rates"]["USD"]

# Adicionando colunas em USD
df["NGR_USD"] = df["NGR"] * exchange_rate
df["Depósitos_USD"] = df["Depósitos"] * exchange_rate

# Filtrando dados por ano (2022, 2023 e ano atual 2024)
df_2022 = df[df['Ano'] == '2022'].copy()
df_2023 = df[df['Ano'] == '2023'].copy()
df_ano_atual = df[df['Ano'] == '2024']

# Somando os valores por ano, incluindo conversão para USD
sum_2022 = {
    'Mês': '2022',
    'NGR': f"R${df_2022['NGR'].sum():,.2f}".replace(",", "X").replace(".", ",").replace("X", "."),
    'Depósitos': f"R${df_2022['Depósitos'].sum():,.2f}".replace(",", "X").replace(".", ",").replace("X", "."),
    'NGR_USD': f"${df_2022['NGR_USD'].sum():,.2f}".replace(",", "X").replace(".", ",").replace("X", "."),
    'Depósitos_USD': f"${df_2022['Depósitos_USD'].sum():,.2f}".replace(",", "X").replace(".", ",").replace("X", "."),
    'Deposit count': df_2022['Deposit count'].sum()
}

sum_2023 = {
    'Mês': '2023',
    'NGR': f"R${df_2023['NGR'].sum():,.2f}".replace(",", "X").replace(".", ",").replace("X", "."),
    'Depósitos': f"R${df_2023['Depósitos'].sum():,.2f}".replace(",", "X").replace(".", ",").replace("X", "."),
    'NGR_USD': f"${df_2023['NGR_USD'].sum():,.2f}".replace(",", "X").replace(".", ",").replace("X", "."),
    'Depósitos_USD': f"${df_2023['Depósitos_USD'].sum():,.2f}".replace(",", "X").replace(".", ",").replace("X", "."),
    'Deposit count': df_2023['Deposit count'].sum()
}

# Calculando o total geral
total_geral = {
    'Mês': 'Total',
    'NGR': f"R${pd.concat([df_2022['NGR'], df_2023['NGR'], df_ano_atual['NGR']]).sum():,.2f}".replace(",", "X").replace(".", ",").replace("X", "."),
    'Depósitos': f"R${pd.concat([df_2022['Depósitos'], df_2023['Depósitos'], df_ano_atual['Depósitos']]).sum():,.2f}".replace(",", "X").replace(".", ",").replace("X", "."),
    'NGR_USD': f"${pd.concat([df_2022['NGR_USD'], df_2023['NGR_USD'], df_ano_atual['NGR_USD']]).sum():,.2f}".replace(",", "X").replace(".", ",").replace("X", "."),
    'Depósitos_USD': f"${pd.concat([df_2022['Depósitos_USD'], df_2023['Depósitos_USD'], df_ano_atual['Depósitos_USD']]).sum():,.2f}".replace(",", "X").replace(".", ",").replace("X", "."),
    'Deposit count': df_2022['Deposit count'].sum() + df_2023['Deposit count'].sum() + df_ano_atual['Deposit count'].sum()
}

# Removendo o ano de cada mês do ano atual (mantendo o formato "MM/AAAA")
df_ano_atual['Mês'] = df_ano_atual['Mês_num'] + '/' + df_ano_atual['Ano']

# Concatenando os resultados e adicionando colunas de conversão para USD
df_final = pd.concat([pd.DataFrame([sum_2022]), pd.DataFrame([sum_2023]), df_ano_atual, pd.DataFrame([total_geral])], ignore_index=True)

# Adicionando a coluna de ordem
df_final['Ordem'] = range(1, len(df_final) + 1)

# Reordenando as colunas para colocar "Ordem" como a primeira coluna
df_final = df_final[['Ordem', 'Mês', 'NGR', 'Depósitos', 'NGR_USD', 'Depósitos_USD', 'Deposit count']]

# Exibindo o resultado final
df_final

C:\Users\Paulo\AppData\Local\Temp\ipykernel_16888\3534059534.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ano_atual['Mês'] = df_ano_atual['Mês_num'] + '/' + df_ano_atual['Ano']


,Ordem,Mês,NGR,Depósitos,NGR_USD,Depósitos_USD,Deposit count
0,1,2022,"R$47.605,08","R$334.029,47","$8.235,68","$57.787,10",3863
1,2,2023,"R$68.871,57","R$256.531,17","$11.914,78","$44.379,89",3764
2,3,01/2024,-973.77,26395.43,-168.46221,4566.40939,467
3,4,02/2024,12485.59,27603.13,2160.00707,4775.34149,501
4,5,03/2024,12516.57,42782.24,2165.36661,7401.32752,454
5,6,04/2024,17007.67,35917.03,2942.32691,6213.64619,594
6,7,05/2024,16921.39,59693.48,2927.40047,10326.97204,946
7,8,06/2024,22373.29,82935.04,3870.57917,14347.76192,1229
8,9,07/2024,27316.54,82117.53,4725.76142,14206.33269,1252
9,10,08/2024,33552.86,101579.48,5804.64478,17573.25004,1578


In [ ]:
# Aplicando a formatação de moeda para o ano atual
df_ano_atual["NGR"] = df_ano_atual["NGR"].apply(lambda x: f'R${x:,.2f}'.replace(",", "X").replace(".", ",").replace("X", "."))
df_ano_atual["Depósitos"] = df_ano_atual["Depósitos"].apply(lambda x: f'R${x:,.2f}'.replace(",", "X").replace(".", ",").replace("X", "."))
df_ano_atual["NGR_USD"] = df_ano_atual["NGR_USD"].apply(lambda x: f'${x:,.2f}'.replace(",", "X").replace(".", ",").replace("X", "."))
df_ano_atual["Depósitos_USD"] = df_ano_atual["Depósitos_USD"].apply(lambda x: f'${x:,.2f}'.replace(",", "X").replace(".", ",").replace("X", "."))

# Concatenando os resultados e adicionando colunas de conversão para USD
df_final = pd.concat([pd.DataFrame([sum_2022]), pd.DataFrame([sum_2023]), df_ano_atual, pd.DataFrame([total_geral])], ignore_index=True)

# Adicionando a coluna de ordem
df_final['Ordem'] = range(1, len(df_final) + 1)

# Reordenando as colunas para colocar "Ordem" como a primeira coluna
df_final = df_final[['Ordem', 'Mês', 'NGR', 'Depósitos', 'NGR_USD', 'Depósitos_USD', 'Deposit count']]

df_final['Data'] = "18/11/2024"


# Exibindo o resultado final
df_final


C:\Users\Paulo\AppData\Local\Temp\ipykernel_16888\1392788.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ano_atual["NGR"] = df_ano_atual["NGR"].apply(lambda x: f'R${x:,.2f}'.replace(",", "X").replace(".", ",").replace("X", "."))
C:\Users\Paulo\AppData\Local\Temp\ipykernel_16888\1392788.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ano_atual["Depósitos"] = df_ano_atual["Depósitos"].apply(lambda x: f'R${x:,.2f}'.replace(",", "X").replace(".", ",").replace("X", "."))
C:\Users\Paulo\AppData\Loc

,Ordem,Mês,NGR,Depósitos,NGR_USD,Depósitos_USD,Deposit count,Data
0,1,2022,"R$47.605,08","R$334.029,47","$8.235,68","$57.787,10",3863,18/11/2024
1,2,2023,"R$68.871,57","R$256.531,17","$11.914,78","$44.379,89",3764,18/11/2024
2,3,01/2024,"R$-973,77","R$26.395,43","$-168,46","$4.566,41",467,18/11/2024
3,4,02/2024,"R$12.485,59","R$27.603,13","$2.160,01","$4.775,34",501,18/11/2024
4,5,03/2024,"R$12.516,57","R$42.782,24","$2.165,37","$7.401,33",454,18/11/2024
5,6,04/2024,"R$17.007,67","R$35.917,03","$2.942,33","$6.213,65",594,18/11/2024
6,7,05/2024,"R$16.921,39","R$59.693,48","$2.927,40","$10.326,97",946,18/11/2024
7,8,06/2024,"R$22.373,29","R$82.935,04","$3.870,58","$14.347,76",1229,18/11/2024
8,9,07/2024,"R$27.316,54","R$82.117,53","$4.725,76","$14.206,33",1252,18/11/2024
9,10,08/2024,"R$33.552,86","R$101.579,48","$5.804,64","$17.573,25",1578,18/11/2024


In [14]:
import uuid

# Carregar o arquivo CSV existente
df_existing = pd.read_excel("daily_playerstats1.xlsx")

# Filtrar as colunas necessárias (incluindo 'ID')
df_existing = df_existing[["Mês", "NGR", "Depósitos", "Deposit count", "NGR_USD", "Depósitos_USD", "Data", "ID", "Ordem"]]

# Gerar um ID único para toda a nova tabela 'final_result'
unique_id = str(uuid.uuid4())

# Adicionar a mesma ID para todas as linhas da nova tabela
df_final['ID'] = unique_id

# Concatenar o DataFrame novo (final_result) ao existente
df_combined = pd.concat([df_existing, df_final], ignore_index=True)

# Remover duplicatas com base na coluna "Username" (ou outra coluna de sua escolha)
#df_combined = df_combined.drop_duplicates(subset=["Ordem"], keep="last")

# Salvar o DataFrame combinado e sem duplicatas no arquivo CSV
df_combined.to_excel("daily_playerstats1.xlsx",index=False)